In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import itertools
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
np.random.seed(101)

# Read in data
cols = ['Date','Price']
df = pd.read_excel('forecasting_take_home_data.xlsx', names=cols, parse_dates=True, skiprows=11)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Visual inspection
plt.figure(figsize=(10,5))
plt.title('Gas prices over time ($)')
plt.xlabel(str(df.columns[0]))
plt.ylabel(str(df.columns[1]))
plt.axhline(y=np.mean(df.iloc[:,1]), color='g', linestyle='--', label='Mean')
plt.legend(loc='upper left')
plt.plot(df[df.columns[0]], df[df.columns[1]])
plt.show()

In [ ]:
def train_test_split(df, months):
    train_df = df.iloc[:-months]
    test_df = df.iloc[-months:]
    return train_df, test_df

def prepare_prophet(df):
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: 'y'})
    return df

def run_cv(df, grid, init, hor):
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    scores = pd.DataFrame()

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial='{} days'.format(init), period='{} days'.format(hor), horizon = '{} days'.format(hor), parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        scores = scores.append(df_p)

    # Find the best parameters
    tuning_results = pd.concat([pd.DataFrame(all_params).reset_index(drop=True), scores.reset_index(drop=True)],axis=1)
    print(tuning_results)
    best_params = all_params[np.argmin(tuning_results['rmse'])]
    return best_params

def predict_months(model, months):
    future = model.make_future_dataframe(periods=months, freq='MS') # Month starting dates
    forecast = model.predict(future)
    fig = model.plot(forecast)
    return forecast

def get_best_forecast(df, months, param_grid, initial, horizon):
    # Finding best parameters over training set using expanding window CV with initial training of 3 years
    # and 1 year horizons (12 predictions for each expansion * (25-3) years = 264 total predictions evaluated)
    # Best parameters are those with lowest RMSE over all 264 predicted to test value comparisons
    params = run_cv(df, param_grid, initial, horizon)

    # Train model and get predictions for test set comparison
    model = Prophet(**params).fit(df)
    forecast = predict_months(model, months)
    return model, forecast

def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))

In [ ]:
# Months to predict into the future
months = 12

# Data prep
df = prepare_prophet(df)
train_df, test_df = train_test_split(df, months)

param_grid = {  
    'changepoint_prior_scale': [0.05, 0.1, 0.5, 1, 10],
    'seasonality_prior_scale': [0.01, 1, 10],
    'changepoint_range': [0.95]
}

model, forecast = get_best_forecast(train_df, months, param_grid, 3*365, 365)

In [ ]:
# Looking at Full Prophet performance graphically
predictions = forecast['yhat'].iloc[-months:]
full_err = rmse(predictions, test_df['y'])

plt.figure(figsize=(10,5))
plt.title("RMSE {}".format(full_err))
plt.ylabel('Price($)')
plt.xlabel('Months since January 1992')
plt.plot(train_df['y'], label='Train set')
plt.plot(test_df['y'], label='Test set')
plt.plot(predictions, label='Prophet Prediction', linestyle='--')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Months to predict into the future
months = 12

# Data prep
df = prepare_prophet(df)
train_df, test_df = train_test_split(df, months)

# Training on just last 8 years
train_df = train_df.loc[(train_df['ds'] >= '2009-01-01')]

# Cross validation search grid
param_grid = {  
    'changepoint_prior_scale': [0.05, 0.1, 0.5, 1, 10],
    'seasonality_prior_scale': [0.01, 1, 10],
    'changepoint_range': [0.95]
}

model, forecast = get_best_forecast(train_df, months, param_grid, 3*365, 365)

In [ ]:
# Getting RMSE of full model w/ best parameters for last 60 month predictions
months=12

df = prepare_prophet(df)
train_df, test_df = train_test_split(df, months)

m = Prophet(changepoint_prior_scale=0.5, seasonality_prior_scale=0.01, changepoint_range= 0.95).fit(train_df)
df_cv = cross_validation(m, initial='1095 days', period='365 days', horizon = '365 days', parallel="processes")

df_cv = df_cv.loc[(df_cv['ds'] >= '2012-02-01')]
print("RMSE of full model for last 5 years: " + str(rmse(df_cv['yhat'], df_cv['y'])))

In [ ]:
# Looking at Subset Prophet performance graphically
predictions = forecast['yhat'].iloc[-months:]
predictions.index = test_df.index
full_err = rmse(predictions, test_df['y'])

train_df, test_df = train_test_split(df, months)

plt.figure(figsize=(10,5))
plt.title("RMSE {}".format(full_err))
plt.ylabel('Price($)')
plt.xlabel('Months since January 1992')
plt.plot(train_df['y'], label='Train set')
plt.plot(test_df['y'], label='Test set')
plt.plot(predictions, label='Subset Prophet', linestyle='--')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Exponential Smoothing

months=12

df = prepare_prophet(df)
train_df, test_df = train_test_split(df, months)

expModel = ExponentialSmoothing(np.array(train_df['y']), trend='add', seasonal='add', seasonal_periods=12).fit()
expForecast = expModel.predict(start=test_df.index[0], end=test_df.index[-1])
expForecast = pd.DataFrame(expForecast, index=test_df.index)

exp_err = rmse(expForecast.iloc[:,0], test_df['y'])

plt.figure(figsize=(10,5))
plt.title("RMSE: {}".format(exp_err))
plt.plot(train_df['y'], label='Train set')
plt.plot(test_df['y'], label='Test set')
plt.plot(expForecast, label='Exponential Smoothing', linestyle='--')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Backtesting Exponential Smoothing
tscv = TimeSeriesSplit(24)

train_df, test_df = train_test_split(df, months)
preds = []
errs = []
for train_index, test_index in tscv.split(train_df):
    #print("TRAIN:", train_index, "TEST:", test_index)
    y_train, y_test = train_df['y'][train_index], train_df['y'][test_index]
    expModel = ExponentialSmoothing(y_train, trend='add', seasonal='add', seasonal_periods=12).fit()
    expForecast = expModel.predict(start=y_test.index[0], end=y_test.index[-1])   
    preds.append(expForecast)
    
    exp_err = rmse(expForecast, y_test)
    errs.append(exp_err)

expModel = ExponentialSmoothing(train_df['y'], trend='add', seasonal='add', seasonal_periods=12).fit()
expForecast = expModel.predict(start=test_df.index[0], end=test_df.index[-1])
preds.append(expForecast)
preds = pd.concat(preds)
average_rmse = np.mean(errs)
print("RMSE over all periods: {}".format(average_rmse))

In [ ]:
plt.figure(figsize=(10,5))
plt.title("RMSE: {}".format(average_rmse))
plt.plot(train_df['y'], label='Train set')
plt.plot(test_df['y'], label='Test set')
plt.plot(preds, label='Exponential Smoothing', linestyle='--')
plt.legend(loc='upper left')
plt.show()